In [ ]:
import json
import sqlite3

import pandas as pd
from tqdm import tqdm

tqdm.pandas()

# UA

Load Ukrainian morphological dictionary<br>
Source of dataset with detailed descriptions here: https://github.com/LinguisticAndInformationSystems/mphdict

## Download dictionary

In [2]:
!curl -o ./data/mph_ua.db https://raw.githubusercontent.com/LinguisticAndInformationSystems/mphdict/master/src/data/mph_ua.db

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 65.7M  100 65.7M    0     0  8711k      0  0:00:07  0:00:07 --:--:-- 9214k


## Read data

In [3]:
sqlite_connection = sqlite3.connect('data/mph_ua.db')

In [4]:
nom = pd.read_sql_query("select * from nom", sqlite_connection)
nom.shape

(261499, 15)

In [5]:
# Wrong characters in the words (digit 6, latin c)
# Filtering them out

cnd = nom.reestr.isin(['П6тік-на-Дубе"цькому', 'ча"cкі', 'ба"нcурі'])
nom = nom.loc[~cnd].copy()
nom.shape

(261496, 15)

In [6]:
nom.sample(2)

,reestr,field2,part,type,field5,field6,field7,digit,nom_old,own,isdel,reverse,isproblem,accent,suppl_accent
7866,"бібельдру""к",0,8,1957,,,,2c27gv6lof,8139,0.0,0,fol6vg72c2,NaN,0.0,NaN
45671,"замаско""вуватися",0,36,9,None,None,None,a1h1mfj3o31nbmx,47015,0.0,0,xmbn13o3jfm1h1a,NaN,0.0,NaN


In [7]:
indents = pd.read_sql_query("select * from indents", sqlite_connection)
indents.shape

(2796, 6)

In [8]:
indents.sample(2)

,type,indent,field3,field4,comment,gr_id
1066,1155,8,0,0,None,5
624,690,5,0,0,None,8


In [9]:
flexes = pd.read_sql_query("select * from flexes", sqlite_connection)
flexes.rename(columns={'field2': 'gramm_category'}, inplace=True)
flexes.shape

(48142, 6)

In [10]:
flexes.sample(2)

,id,flex,gramm_category,xmpl,type,digit
44591,49799,осами,12.0,None,2843,jm1hb
6138,6953,уй,2.0,None,380,oe


In [11]:
gr = pd.read_sql_query("select * from gr", sqlite_connection)
gr.shape

(17, 31)

In [12]:
gr.head(2)

,id,part_of_speech,field4,field5,field6,field7,field8,field9,field10,field11,...,field23,field24,field25,field26,field27,field28,field29,field30,field31,field32
0,0,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,1,іменник,Н s,Р s,Д s,З s,О s,М s,К s,Н p,...,None,None,None,None,None,None,None,None,None,None


In [13]:
parts = pd.read_sql_query("select * from parts", sqlite_connection)
parts.shape

(68, 10)

In [14]:
parts.head(2)

,id,part,com,ac,gr_id,rid,mnozh,istota,vid,adjekt
0,5,ж,іменник жіночого роду,None,1.0,2,0,2,0,0
1,6,жі,"іменник жіночого роду, істота",None,1.0,2,2,1,0,0


## Wordforms generation

In [15]:
nom_parts = pd.merge(nom[['reestr', 'type', 'part', 'field5', 'field6', 'field7']].reset_index().rename(columns={'index': 'word_base_id'}),
                     parts[['id', 'com']],
                     left_on='part',
                     right_on='id',
                    )
nom_parts.drop(columns=['id'], inplace=True)
nom_parts.rename(columns={'part': 'pos_id', 'com': 'pos_name'}, inplace=True)
nom_parts.sort_values(by='word_base_id', inplace=True)
nom.shape, nom_parts.shape

((261496, 15), (261496, 8))

In [16]:
nom_parts.sample(4)

,word_base_id,reestr,type,pos_id,field5,field6,field7,pos_name
225670,192902,"Подоляки""",2225,14,(населений пункт в Україні),None,None,множинний іменник
166301,214492,"облистві""ння",2108,13,None,None,None,іменник середнього роду
29378,242277,"Ме""лець",1547,8,(місто в Польщі),None,None,іменник чоловічого роду
232512,42163,"забрести""",112,35,None,None,None,дієслово доконаного виду


In [17]:
nom_indents = pd.merge(nom_parts,
                       indents[['type', 'indent']],
                       on='type'
                      )
nom_parts.shape, nom_indents.shape

((261496, 8), (261496, 9))

In [18]:
nom_indents.sample(4)

,word_base_id,reestr,type,pos_id,field5,field6,field7,pos_name,indent
196421,211863,"картелюва""тися",18,36,None,None,None,дієслово недоконаного виду,6
135904,139289,"розледачі""лий",2342,10,None,None,None,дієприкметник,2
187390,253268,"одбува""ти",697,36,,,,дієслово недоконаного виду,2
103606,252035,"аро""нія",1182,5,,,,іменник жіночого роду,1


In [19]:
nom_indents['reestr'] = nom_indents.reestr.str.replace('"', '').str.replace('#', '')

In [20]:
# Remove records with spaces in them
# TODO: revisit those words
cnd = ~nom_indents.reestr.apply(lambda x: ' ' in x)
print(f'Removed {(~cnd).sum()} words')
nom_indents = nom_indents.loc[cnd].copy()
nom_indents.shape

Removed 816 words


(260680, 9)

In [21]:
nom_indents.sample(4)

,word_base_id,reestr,type,pos_id,field5,field6,field7,pos_name,indent
67305,28923,глютамін,1864,8,None,None,None,іменник чоловічого роду,0
158786,29100,гнояка,1428,5,None,[зневажл.],None,іменник жіночого роду,2
122763,156955,сфальшування,2108,13,None,None,None,іменник середнього роду,1
87800,71279,малопоживний,2324,11,None,None,None,прикметник,2


In [22]:
nom_indents['word_base'] = nom_indents.apply(lambda x: x.reestr[:len(x.reestr)-x.indent], axis=1)

In [23]:
nom_indents.sort_values(by='word_base_id', inplace=True)

In [24]:
nom_indents.sample(4)

,word_base_id,reestr,type,pos_id,field5,field6,field7,pos_name,indent,word_base
257130,245423,Пії,2185,14,(населений пункт в Україні),None,None,множинний іменник,1,Пі
236392,48668,затикатися,702,35,(заквітчатися),None,None,дієслово доконаного виду,4,затика
163311,240891,Линтварівка,1428,5,(населений пункт в Україні),,,іменник жіночого роду,2,Линтварів
30788,142908,рубіновий,2302,11,None,None,None,прикметник,2,рубінов


In [25]:
nom_flexes = pd.merge(nom_indents,
                      flexes[['flex', 'gramm_category', 'type']],
                      on='type'
                     )
nom_flexes.shape

(4929115, 12)

In [26]:
cnd = nom_flexes.flex.isna() | nom_flexes.flex.isin(['empty_', ' '])
nom_flexes.loc[cnd, 'flex'] = ''

In [27]:
# Fix broken style
cnd = nom_flexes.flex.str.startswith('^')
nom_flexes.loc[cnd, 'flex'] = nom_flexes.loc[cnd, 'flex'].apply(lambda x: x[1:] + '^')

In [28]:
nom_flexes['word'] = (nom_flexes.word_base + nom_flexes.flex).str.lower()

In [29]:
cnd = ~nom_flexes.word.apply(lambda x: any(symbol in x for symbol in ' ()'))
nom_flexes = nom_flexes.loc[cnd].copy()
nom_flexes.shape

(4928901, 13)

In [30]:
nom_flexes.sample(4)

,word_base_id,reestr,type,pos_id,field5,field6,field7,pos_name,indent,word_base,flex,gramm_category,word
3046726,13243,вертольотобудівник,1788,7,None,None,None,"іменник чоловічого роду, істота",0,вертольотобудівник,а,2.0,вертольотобудівника
1223211,202067,ропчанський,2326,11,None,None,None,прикметник,2,ропчанськ,ім,18.0,ропчанськім
4672027,123061,поцілитися,1098,35,None,None,None,дієслово доконаного виду,5,поціл,ивсь,11.0,поціливсь
816889,169139,фейлетончик,1776,8,None,None,None,іменник чоловічого роду,0,фейлетончик,ах,13.0,фейлетончиках


In [31]:
nom_flexes['flex_note'] = float('nan')

with tqdm(total=6) as tbar:
    cnd = nom_flexes.word.str[-1] == '^'
    nom_flexes.loc[cnd, 'flex_note'] = "specific, shouldn't be used"
    nom_flexes.loc[cnd, 'word'] = nom_flexes.loc[cnd, 'word'].str[:-1]
    tbar.update(1)

    cnd = nom_flexes.word.str[-1] == '*'
    nom_flexes.loc[cnd, 'flex_note'] = "rare form"
    nom_flexes.loc[cnd, 'word'] = nom_flexes.loc[cnd, 'word'].str[:-1]
    tbar.update(1)

    cnd = nom_flexes.word.str[-1] == '%'
    nom_flexes.loc[cnd, 'flex_note'] = 'Used like "по ..."'
    nom_flexes.loc[cnd, 'word'] = nom_flexes.loc[cnd, 'word'].str[:-1]
    tbar.update(1)

    cnd = nom_flexes.word.str[-1] == '$'
    nom_flexes.loc[cnd, 'flex_note'] = 'Used like "на ..."'
    nom_flexes.loc[cnd, 'word'] = nom_flexes.loc[cnd, 'word'].str[:-1]
    tbar.update(1)

    cnd = nom_flexes.word.str[-1] == '&'
    nom_flexes.loc[cnd, 'flex_note'] = 'Used like "у/на ..."'
    nom_flexes.loc[cnd, 'word'] = nom_flexes.loc[cnd, 'word'].str[:-1]
    tbar.update(1)

    cnd = nom_flexes.word.str[-1] == '@'
    nom_flexes.loc[cnd, 'flex_note'] = 'Used like "до ..."'
    nom_flexes.loc[cnd, 'word'] = nom_flexes.loc[cnd, 'word'].str[:-1]
    tbar.update(1)

100%|█████████████████████████████████████████████████████████████████| 6/6 [00:29<00:00,  4.87s/it]


In [32]:
nom_flexes.sort_values(by=['word_base_id', 'gramm_category'], inplace=True)

In [33]:
nom_flexes.reset_index(drop=True, inplace=True)

In [34]:
nom_flexes.reset_index(inplace=True)
nom_flexes.rename(columns={'index': 'flex_id'}, inplace=True)

In [35]:
nom_flexes.sample(4)

,flex_id,word_base_id,reestr,type,pos_id,field5,field6,field7,pos_name,indent,word_base,flex,gramm_category,word,flex_note
422927,422927,21558,вікунья,1222,6,(тварина),None,None,"іменник жіночого роду, істота",2,вікун,ья,1.0,вікунья,NaN
320354,320354,16771,висип,1853,8,None,None,None,іменник чоловічого роду,0,висип,и*,14.0,висипи,rare form
10369,10369,556,автометричний,2302,11,None,None,None,прикметник,2,автометричн,ій,9.0,автометричній,NaN
3863018,3863018,198874,кізлівський,2326,11,None,None,None,прикметник,2,кізлівськ,ая^,7.0,кізлівськая,"specific, shouldn't be used"


In [36]:
def flex_to_dict(record):
    ans = {
        'wordform': record.word,
        'main_form': record.reestr.lower(),
        'mphdict_word_base_id': record.word_base_id,
        'mphdict_pos_name': record.pos_name,
        'mphdict_gramm_category': record.gramm_category,
        'mphdict_field5': record.field5,
    }
    return ans

all_wordforms = nom_flexes.progress_apply(flex_to_dict, axis=1).tolist()
all_wordforms[:3]

100%|██████████████████████████████████████████████████| 4928901/4928901 [05:40<00:00, 14486.56it/s]


[{'wordform': 'а',
  'main_form': 'а',
  'mphdict_word_base_id': 0,
  'mphdict_pos_name': 'вигук',
  'mphdict_gramm_category': nan,
  'mphdict_field5': None},
 {'wordform': 'а',
  'main_form': 'а',
  'mphdict_word_base_id': 1,
  'mphdict_pos_name': 'сполучник',
  'mphdict_gramm_category': nan,
  'mphdict_field5': None},
 {'wordform': 'а',
  'main_form': 'а',
  'mphdict_word_base_id': 2,
  'mphdict_pos_name': 'частка',
  'mphdict_gramm_category': nan,
  'mphdict_field5': None}]

## Wordforms postprocessing

In [37]:
# remove NaN fields
for i in tqdm(all_wordforms):
    for k, v in list(i.items()):
        if pd.isna(v):
            del i[k]

100%|█████████████████████████████████████████████████| 4928901/4928901 [00:22<00:00, 220785.45it/s]


In [38]:
for i in tqdm(all_wordforms):
    i['lang'] = 'українська'
    gc = i.get('mphdict_gramm_category')
    if isinstance(gc, float):
        i['mphdict_gramm_category'] = int(gc)

100%|████████████████████████████████████████████████| 4928901/4928901 [00:04<00:00, 1085908.46it/s]


In [39]:
all_wordforms[:3]

[{'wordform': 'а',
  'main_form': 'а',
  'mphdict_word_base_id': 0,
  'mphdict_pos_name': 'вигук',
  'lang': 'українська'},
 {'wordform': 'а',
  'main_form': 'а',
  'mphdict_word_base_id': 1,
  'mphdict_pos_name': 'сполучник',
  'lang': 'українська'},
 {'wordform': 'а',
  'main_form': 'а',
  'mphdict_word_base_id': 2,
  'mphdict_pos_name': 'частка',
  'lang': 'українська'}]

### Surnames

In [40]:
cases = 'називний родовий давальний знахідний орудний місцевий кличний'.split()

j = 0
for i in tqdm(all_wordforms):
    if 'прізвище' in i.get('mphdict_pos_name'):
        i['pos'] = 'proper_name'
        i['person_name_part'] = 'surname'
        gramm_category = i.get('mphdict_gramm_category', 100)
        if gramm_category < 22:
            i['case_ukr'] = cases[(gramm_category-1) % 7]

100%|████████████████████████████████████████████████| 4928901/4928901 [00:02<00:00, 2266606.42it/s]


## Further postprocessing

In [41]:
case_ukr_to_case = {
    'називний': 'nominative',
    'родовий': 'genitive',
    'давальний': 'dative',
    'знахідний': 'accusative',
    'орудний': 'instrumental',
    'місцевий': 'locative',
    'кличний': 'vocative'
}

for i in tqdm(all_wordforms):
    if 'case_ukr' in i:
        i['case'] = case_ukr_to_case[i['case_ukr']]

100%|████████████████████████████████████████████████| 4928901/4928901 [00:01<00:00, 2857517.33it/s]


## Saving

In [42]:
len(all_wordforms)

4928901

In [43]:
with open('data/ukr_wordforms.json', 'w') as f:
    json.dump(all_wordforms, f, indent=2)

# RU

Processing was implemented for dataset from that website: http://odict.ru/ <br>
As specified on the website - that dataset changed distribution terms since I worked with it last time, so next steps for preparing Russian language wordforms are no more relevant

## Read data

In [45]:
odict = pd.read_csv('data/odict_resaved.csv', header=None, encoding='windows-1251')
odict.shape

/home/alex/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (214,215,216,217) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(103358, 218)

## Wordforms generation

In [47]:
odict = odict.progress_apply(lambda row: row.apply(lambda x: x.lower() if isinstance(x, str) else x)).copy()

100%|█████████████████████████████████████████████████████████████| 218/218 [00:06<00:00, 32.16it/s]


In [48]:
odict.drop_duplicates(inplace=True)
odict.shape

(102098, 218)

In [49]:
# All POS-like tags
odict[1].value_counts()

п          21402
ж          15203
м          14299
св         14006
нсв        12150
мо          9415
с           6475
жо          3821
н           1453
мн.         1021
св-нсв       905
ф.           440
мо-жо        423
предик.      280
межд.        185
предл.       127
част.        108
союз          87
вводн.        65
числ.         63
мс-п          50
со            42
сравн.        39
числ.-п       39
Name: 1, dtype: int64

In [50]:
cases = ['именительный', 'родительный', 'дательный', 'винительный', 'творительный', 'предложный']

wordforms = []

def account_row(row):
    word = {
        'lang': 'русский',
        'source': 'odict',
        'main_form': row.iloc[0],
    }
    odict_pos = row.iloc[1]
    row = row.drop([1]).dropna().copy()
    for i, v in row.iteritems():
        # wf = word.copy()
        wf = dict(**word)
        wf |= {
            'wordform': v,
            'odict_column': i,
            'odict_row': row.name,
            'odict_pos': odict_pos
        }
        if odict_pos == 'ф.':  # surnames
            wf['pos'] = 'proper_name'
            wf['person_name_part'] = 'surname'
            if i > 0:
                i -= 1
            if i < 21:
                case_idx = i % 7
                if case_idx < 6:
                    wf['case_ru'] = cases[case_idx]
        if 15 in row and row[15].endswith('вич'):  # male name + patronimic
            wf['pos'] = 'proper_name'
            if i < 15:
                wf['person_name_part'] = 'first_name'
            else:
                wf['person_name_part'] = 'patronimic'
                wf['father_name'] = row.iloc[0]
                if i < 29:
                    wf['main_form'] = row[15]
                elif i < 44:
                    wf['main_form'] = row[29]
                elif i < 58:
                    wf['main_form'] = row[44]
                else:
                    wf['main_form'] = row[58]
        wordforms.append(wf)

odict.progress_apply(account_row, axis=1)
len(wordforms)

100%|██████████████████████████████████████████████████████| 102098/102098 [02:04<00:00, 821.69it/s]


4449609

In [54]:
case_ru_to_case = {
    'именительный': 'nominative',
    'родительный': 'genitive',
    'дательный': 'dative',
    'винительный': 'accusative',
    'творительный': 'instrumental',
    'предложный': 'locative'
}

for i in tqdm(wordforms):
    if 'case_ru' in i:
        i['case'] = case_ru_to_case[i['case_ru']]

100%|████████████████████████████████████████████████| 4449609/4449609 [00:01<00:00, 2843901.54it/s]


In [55]:
wordforms[:3]

[{'lang': 'русский',
  'source': 'odict',
  'main_form': 'а',
  'wordform': 'а',
  'odict_column': 0,
  'odict_row': 0,
  'odict_pos': 'межд.'},
 {'lang': 'русский',
  'source': 'odict',
  'main_form': 'а',
  'wordform': 'а',
  'odict_column': 0,
  'odict_row': 1,
  'odict_pos': 'с'},
 {'lang': 'русский',
  'source': 'odict',
  'main_form': 'а',
  'wordform': 'а',
  'odict_column': 2,
  'odict_row': 1,
  'odict_pos': 'с'}]

In [56]:
# Filter out problematic wordforms

def is_wf_ok(wf):
    if wf['wordform'].startswith('-'):
        return False
    return True


wordforms = [i for i in wordforms if is_wf_ok(i)]
len(wordforms)

4449602

## Saving

In [57]:
len(wordforms)

4449602

In [58]:
with open('data/ru_wordforms.json', 'w') as f:
    json.dump(wordforms, f, indent=2)